# Importing python libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import dateutil.parser as dparser

# Reading dataset

###### Load loan.csv in a dataframe

In [2]:
# Specify columns which need to be parsed as dates
parse_dates = ["last_pymnt_d", "next_pymnt_d"]

In [3]:
loan_df = pd.read_csv("loan.csv", parse_dates = parse_dates)

# Understanding dataset

### Data overview

In [4]:
loan_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [5]:
loan_df.shape

(39717, 111)

In [6]:
pd.value_counts(loan_df.dtypes)

float64    74
object     24
int64      13
dtype: int64

In [7]:
loan_df.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 111 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    id                              39717 non-null  int64  
 1    member_id                       39717 non-null  int64  
 2    loan_amnt                       39717 non-null  int64  
 3    funded_amnt                     39717 non-null  int64  
 4    funded_amnt_inv                 39717 non-null  float64
 5    term                            39717 non-null  object 
 6    int_rate                        39717 non-null  object 
 7    installment                     39717 non-null  float64
 8    grade                           39717 non-null  object 
 9    sub_grade                       39717 non-null  object 
 10   emp_title                       37258 non-null  object 
 11   emp_length                      38642 non-null  object 
 12   home_ownership  

#### Analysing columns with all null values

In [8]:
columns_with_no_data = loan_df.columns[loan_df.isnull().all()].to_list()

In [9]:
len(columns_with_no_data)

54

There are total 54 columns out of total 111, with all null values in the dataset. We can get rid of them as they won't bring any value to our analysis.

# Data cleaning

### Remove columns having 0 non-null values

In [10]:
loan_df.drop(columns = columns_with_no_data, inplace = True)
loan_df.shape

(39717, 57)

### Removing columns with more than 30% of null values

In [11]:
null_values_threshold = int(loan_df.shape[0] * (30 / 100))
null_values_threshold

11915

In [12]:
columns_with_highers_null_values = loan_df.columns[loan_df.isnull().sum() > null_values_threshold].to_list()
columns_with_highers_null_values

['desc', 'mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d']

##### Columns having more than 30% missing values would be difficult to impute and might lead to wrong results so we can drop them

In [13]:
loan_df.drop(columns = columns_with_highers_null_values, inplace = True)
loan_df.shape

(39717, 53)

#### Removing column zip_code as it is a masked data

In [14]:
loan_df.drop("zip_code", axis = 1, inplace = True)
loan_df.shape

(39717, 52)

#### Removing url as its derived from id column and doesn't serve any special purpose for analysis

In [15]:
loan_df.drop("url", axis = 1, inplace = True)
loan_df.shape

(39717, 51)

#### Removing emp_title. Seems to be just some company names of customers

In [16]:
loan_df.drop("emp_title", axis = 1, inplace = True)
loan_df.shape

(39717, 50)

#### Removing columns tax_liens and delinq_amnt because all values are either null or 0

In [17]:
loan_df[["tax_liens", "delinq_amnt"]].describe().T

,count,mean,std,min,25%,50%,75%,max
tax_liens,39678.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
delinq_amnt,39717.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
loan_df.drop(columns = ["tax_liens", "delinq_amnt"], inplace = True)
loan_df.shape

(39717, 48)

#### Understanding loan_status

In [19]:
loan_df["loan_status"].value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

We need to dropoff "Current" loan_status from the dataset because we can not do a prescriptive analysis on the ongoing loans

In [20]:
loan_df = loan_df[~(loan_df["loan_status"] == "Current")]
loan_df.shape

(38577, 48)

#### id and member_id are identifiers so we can drop these

In [21]:
print("is id column an identifier: " + str(loan_df["id"].is_unique))
print("is member_id column an identifier: " + str(loan_df["member_id"].is_unique))

is id column an identifier: True
is member_id column an identifier: True


In [23]:
loan_df.drop(columns = ["id", "member_id"], inplace = True)
loan_df.shape

(38577, 46)

#### Inspect application_type

In [26]:
loan_df["application_type"].value_counts()

INDIVIDUAL    38577
Name: application_type, dtype: int64

In [27]:
# Droping as all application_type are same
loan_df.drop("application_type", axis = 1, inplace = True)
loan_df.shape

(38577, 45)

#### Duplicate data inspection

In [24]:
loan_df.duplicated().any()

False

In [22]:
loan_df.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38577 entries, 0 to 39716
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          38577 non-null  int64  
 1   member_id                   38577 non-null  int64  
 2   loan_amnt                   38577 non-null  int64  
 3   funded_amnt                 38577 non-null  int64  
 4   funded_amnt_inv             38577 non-null  float64
 5   term                        38577 non-null  object 
 6   int_rate                    38577 non-null  object 
 7   installment                 38577 non-null  float64
 8   grade                       38577 non-null  object 
 9   sub_grade                   38577 non-null  object 
 10  emp_length                  37544 non-null  object 
 11  home_ownership              38577 non-null  object 
 12  annual_inc                  38577 non-null  float64
 13  verification_status         385